# Decoding a Sequence

**So far we have seen how to train a HMM.**

**Now we will focus on, once trained, how to make predictions efficiently with a HMM**


Given the learned parameters and a new
observation sequence $x = x_1\ldots x_N$, we want to find the sequence of hidden states $y^* = y_1^* \ldots y_N^*$ that "best" explains it.
 This is called the **decoding problem**. 
 
 There are several ways to define what we mean by the "best" $y^*$, depending on our goal: for instance, we may want to minimize the probability of error on each hidden
variable $Y_i$ (posterior decoding), or we may want to find the best assignment to the sequence $Y_1\ldots Y_N$ as a whole (viterbi decoding). 
Therefore, finding the best sequence
can be accomplished through different approaches:

- ** posterior decoding** or **minimum risk decoding**

    This approach selects, at each step $i$, the state that maximizes the conditional probability of $Y_i$ given all the visible sequence. Notice that the state sequence that this approach reaches is not necesary the one that maximizes the probability of the whole sequence and state sequence.
    
\begin{equation}
y_i^* = \arg \max_{y_i \in \Lambda} P(Y_i=y_i | X_1=x_1,\ldots,X_N =x_N).
\end{equation}

- ** Viterbi decoding**

    This approach choose the sequence of states that, overall, has the highest probability.

\begin{eqnarray}
y^* &=& \text{argmax}_{y = y_1\ldots y_N} P(Y_1=y_1,\ldots, Y_N=y_N | X_1=x_1,\ldots,X_N =x_N)\nonumber\\
&=& \text{argmax}_{y = y_1\ldots y_N} P(Y_1=y_1,\ldots, Y_N=y_N, X_1=x_1,\ldots,X_N =x_N).
\end{eqnarray}

### Unfolding all state sequences: trellis representation
Both previous approaches, viterbi decoding and posterior decoding, rely on dynamic programming and make use of the
independence assumptions of the HMM model. Moreover, they use an alternative representation of the HMM called a trellis. 

A trellis unfolds all possible states for each position and it makes explicit the independence assumption: each position only
depends on the previous position. Here, each column represents a position in the sequence and each row represents a possible state. The following figure shows the trellis for $x = \text{walk walk shop clean}$


<img src="../images_for_notebooks/day_2/hmm_trellis.png" style="max-width:100%; width: 60%">



Considering the trellis representation, note that we can include the following information:
- an initial probability to the arrows that depart from the start symbol;
- a final probability to the arrows that reach the stop symbol
- a transition probability to the remaining arrows
-  an emission probability to each circle, which is the probability that the observed symbol is emitted by that particular state.


###  Posterior decoding

picking the highest state posterior for each position $i$ in the sequence:

\begin{equation}
y_i^* = \arg \max_{y_i \in \Lambda} P(Y_i=y_i | X_1=x_1,\ldots,X_N =x_N).
\end{equation}
 
Note, however, that this approach does not guarantee that the sequence $y^*=y_1^* \ldots y_N^*$ will be a
valid sequence of the model. For instance, there might be a transition
between two of the best state posteriors with probability zero. 

### Viterbi decoding

consists in
picking the best global hidden state sequence: 

\begin{eqnarray}
y^* &=& \text{argmax}_{y = y_1\ldots y_N} P(Y_1=y_1,\ldots, Y_N=y_N | X_1=x_1,\ldots,X_N =x_N)\nonumber\\
&=& \text{argmax}_{y = y_1\ldots y_N} P(Y_1=y_1,\ldots, Y_N=y_N, X_1=x_1,\ldots,X_N =x_N).
\end{eqnarray}


## Viterbi decoding

### Working with scores not probabilities

For convenience, we will be working with 
log-probabilities, rather than probabilities. Therefore, if we associate to each circle and arrow in the trellis a score that corresponds
to the log-probabilities above, and if we define the score of a path
connecting the ${\tt start}$ and  ${\tt stop}$ symbols as
the sum of the scores of the circles and arrows it traverses, 
then the goal of **finding the most likely sequence of states (Viterbi decoding) corresponds to finding the path with the highest score**.



The trellis scores are given by the following expressions:

- For each state $c_k$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{init}}(c_k) &=&
\log P_{\mathrm{init}}(Y_{1} = c_k | \text{start}).
\end{eqnarray}


- For each position $i \in {1,\ldots,N-1}$ and each pair of states $c_k$ and $c_l$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{trans}}(i, c_k, c_l) &=&
\log P_{\mathrm{trans}}(Y_{i+1} = c_k | Y_i = c_l).
\end{eqnarray}


- For each state $c_l$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{final}}(c_l) &=&
\log P_{\mathrm{final}}(\text{stop} | Y_N = c_l).
\end{eqnarray}


- For each position $i \in {1,\ldots,N}$ and state $c_k$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{emiss}}(i, c_k) &=&
\log P_{\mathrm{emiss}}(X_i = x_i | Y_i = c_k).
\end{eqnarray}



#### The score of a path in the trellis is equivalent to the log-probability log P(x, y)

Since the joint distribution $P_{\theta} (X=x^m,Y=y^m)$ is given by the formula 

$$
P(x,y)= 
P_{\mathrm{init}}(y_1|\text{start}) 
\left(
\prod_{i=1}^{N-1} P_{\mathrm{trans}}(y_{i+1}|y_i)
\right)
P_{\mathrm{final}}(\text{stop}|y_N)
\prod_{i=1}^{N} P_{\mathrm{emiss}}(x_i|y_i)
$$

when we apply the logarithm we get a sum of logarithms of 4 terms. Using the score notation defined above we get


$$
\log P(x,y)= \mathrm{score}_{\mathrm{init}}(y_1) + \sum_{i=1}^{N-1}\mathrm{score}_{\mathrm{trans}}(i, y_i, y_{i-1}) +
\mathrm{score}_{\mathrm{final}}(c_l) +
 \sum_{i=1}^{N} \mathrm{score}_{\mathrm{emiss}}(i, y_k) 
$$

Since a path in the trellis is just an assignment of states $y=y_1,\dots,y_N$ given words $x=x_1,\dots,x_N$, computing the score of a path is just the sum of scores above. Moreover we have seen this is equivalent to computing the log probability of $(x,y)$.

##  Exercise 2.3

Convince yourself that the score of a path in the trellis (summing over the scores above) is equivalent to the log-probability 
log P(X = x, Y = y), as defined in Eq. 2.2. Use the given function compute scores on the first training sequence and confirm
that the values are correct.
You should get the same values as presented below.

** Suggestion: use an example of length 5 instead of 4, emission_scores is a matrix of n_rows=len(sequence)**


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
# We will this append to ensure we can import lxmls toolking
sys.path.append('../../lxmls-toolkit')

In [68]:
import lxmls.sequences.hmm as hmmc
import lxmls.readers.simple_sequence as ssr
simple = ssr.SimpleSequence()

hmm = hmmc.HMM(simple.x_dict, simple.y_dict)
hmm.train_supervised(simple.train)

In [40]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[1])

print "initial_scores;\n", initial_scores, "\n"
print "transition_scores: \n",transition_scores, "\n"
print "final_scores:\n", final_scores, "\n"
print "emission_scores:\n", emission_scores, "\n"

initial_scores;
[-0.40546511 -1.09861229] 

transition_scores: 
[[[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]] 

final_scores:
[       -inf -0.98082925] 

emission_scores:
[[-0.28768207 -1.38629436]
 [-0.28768207 -1.38629436]
 [-1.38629436 -0.98082925]
 [       -inf -0.98082925]] 



../../lxmls-toolkit/lxmls/sequences/hmm.py:194: RuntimeWarning: divide by zero encountered in log
  transition_scores[pos-1, :, :] = np.log(self.transition_probs)
../../lxmls-toolkit/lxmls/sequences/hmm.py:192: RuntimeWarning: divide by zero encountered in log
  emission_scores[pos, :] = np.log(self.emission_probs[sequence.x[pos], :])
../../lxmls-toolkit/lxmls/sequences/hmm.py:197: RuntimeWarning: divide by zero encountered in log
  final_scores = np.log(self.final_probs)


### Notice a couple of things:

- transition_scores is a matrix of shape (3,2,2), the first dimension corresponds to the len(x)-1
    - The same matrix at each position is copied since the HMM is homogeneous 

In [41]:
simple.train.seq_list[1]

walk/rainy walk/rainy shop/rainy clean/sunny 

In [42]:
print "Emission Probabilities\n", hmm.emission_probs

Emission Probabilities
[[ 0.75   0.25 ]
 [ 0.25   0.375]
 [ 0.     0.375]
 [ 0.     0.   ]]


In [20]:
print "transition Probabilities\n", hmm.transition_probs

transition Probabilities
[[ 0.5    0.   ]
 [ 0.5    0.625]]


In [21]:
transition_scores.shape

NameError: name 'transition_scores' is not defined

- **if emission_scores = log (emission_probabilities) why are there not 3 -inf????**
- **Why we save length(x)-1 times the transition_scores??**


    def compute_scores(self, sequence):
        length = len(sequence.x) # Length of the sequence.
        num_states = self.get_num_states() # Number of states of the HMM.

        # Initial position.
        initial_scores = np.log(self.initial_probs)

        # Intermediate position.
        # logzero is just -np.inf
        emission_scores = np.zeros([length, num_states]) + logzero()
        transition_scores = np.zeros([length-1, num_states, num_states]) + logzero()
        for pos in xrange(length):
            import pdb;pdb.set_trace()
            emission_scores[pos,:] = np.log(self.emission_probs[sequence.x[pos], :])
            if pos > 0:
                transition_scores[pos-1,:,:] = np.log(self.transition_probs)

        # Final position.
        final_scores = np.log(self.final_probs)

        return initial_scores, transition_scores, final_scores, emission_scores
        
        
Could be changed to


    def compute_scores(self, sequence):
        length = len(sequence.x) # Length of the sequence.
        num_states = self.get_num_states() # Number of states of the HMM.

        # Initial position.
        initial_scores = np.log(self.initial_probs)

        # Intermediate positions
        transition_scores = np.log(self.transition_probs) ## now we don't copy the matrix per position
        emission_scores = np.log(self.emission_probs[sequence.x,:])
        
        # Final position.
        final_scores = np.log(self.final_probs)

        return initial_scores, transition_scores, final_scores, emission_scores
        

## Exercise 2.4 

Look at the module ``sequences/log_domain.py.`` This module implements a function ```logsum_pair(logx, logy)``` to add two numbers
represented in the log-domain; it returns their sum also represented in the log-domain.

The function ```logsum(logv)``` sums all components of an array represented in the log-domain.
This will be used later in our decoding algorithms. To observe why this is important, type the following:

In [22]:
#help(np.log1p)

In [23]:
import numpy as np

#a = np.random.rand([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])
a = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])

print np.log(sum(np.exp(a)))
print np.log(sum(np.exp(10*a)))
print np.log(sum(np.exp(100*a)))
print np.log(sum(np.exp(1000*a)))

print "\n"
from lxmls.sequences.log_domain import logsum
print logsum(a)
print logsum(10*a)
print logsum(100*a)
print logsum(1000*a)

2.79990988434
9.45876378448
90.000045401
inf


2.79990988434
9.45876378448
90.000045401
900.0


/Users/macpro/anaconda/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: overflow encountered in exp


In [24]:

def logzero():
    return -np.inf


def safe_log(x):
    print(x)
    if x == 0:
        return logzero()
    return np.log(x)


def logsum_pair(logx, logy):
    """
    Return log(x+y), avoiding arithmetic underflow/overflow.

    logx: log(x)
    logy: log(y)

    Rationale:

    x + y    = e^logx + e^logy
             = e^logx (1 + e^(logy-logx))
    log(x+y) = logx + log(1 + e^(logy-logx)) (1)

    Likewise,
    log(x+y) = logy + log(1 + e^(logx-logy)) (2)

    The computation of the exponential overflows earlier and is less precise
    for big values than for small values. Due to the presence of logy-logx
    (resp. logx-logy), (1) is preferred when logx > logy and (2) is preferred
    otherwise.
    """
    if logx == logzero():
        return logy
    elif logx > logy:
        return logx + np.log1p(np.exp(logy-logx))
    else:
        return logy + np.log1p(np.exp(logx-logy))


def logsum(logv):
    """
    Return log(v[0]+v[1]+...), avoiding arithmetic underflow/overflow.
    """
    res = logzero()
    for val in logv:
        res = logsum_pair(res, val)
    return res